In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Factory

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
pool_created_filt = Filter.create_filter(address=None, event_types=[Pair.events.PoolCreated])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=pool_created_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, token0, token1, fee = evt["topics"]
    args = Conversion().decode_data(evt["data"])

    token0_address = Conversion().convert_uint256_string_to_address(token0)
    token1_address = Conversion().convert_uint256_string_to_address(token1)
    token0 = Token().fetch_erc20_details(web3, token0_address, raise_on_error=False)
    token1 = Token().fetch_erc20_details(web3, token1_address, raise_on_error=False)
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address
    event['details']['token0_symbol'] = token0.symbol
    event['details']['token1_symbol'] = token1.symbol  
    event['details']['fee'] = Conversion().convert_uint256_string_to_int(fee) 
 
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:61,381,760 tx:0x719ed60496788b0ee7726bb370ed041b72820bfb7a33d3fcdb54ece3612b72a2
Swap at block:61,383,670 tx:0x3f6f4f30ee4e0efc203618ec0d36d6bff0ca0d2da0301f6361dd5ae84132822a
Swap at block:61,383,846 tx:0x2e71a026d0ac0a5072efefc4599c14cd337badab1f1c0f2299607f098bcf6a55
Swap at block:61,384,080 tx:0x1ca359302cf069287d16cca3976cfbaea8fc1ca351bf2d82ac0e8c29088b5c45
Swap at block:61,384,612 tx:0x0dd94b0324932858aaccaecaa2f1ebb336a2e098d535224067845a09d933941e
Swap at block:61,386,834 tx:0xa41c907dedcee7d8e4f59ce53c3823aa6ec4e73fc15cf569d7a4ca3499b8a951
Swap at block:61,388,160 tx:0x2b0a866363b4aabfe76f6c5e679e74aa8b61ce247c85a5679ea5d77051656be3
Swap at block:61,388,525 tx:0x197ac382d39ce862d987979bbcc9a7533e64523e5c2818c76c3ca3ecac53b04e
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x719ed60496788b0ee7726bb370ed041b72820bfb7a33...,61381760,1725380635,{'web3_type': <class 'web3._utils.datatypes.Po...
1,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x3f6f4f30ee4e0efc203618ec0d36d6bff0ca0d2da030...,61383670,1725384795,{'web3_type': <class 'web3._utils.datatypes.Po...
2,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x2e71a026d0ac0a5072efefc4599c14cd337badab1f1c...,61383846,1725385169,{'web3_type': <class 'web3._utils.datatypes.Po...
3,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x1ca359302cf069287d16cca3976cfbaea8fc1ca351bf...,61384080,1725385695,{'web3_type': <class 'web3._utils.datatypes.Po...
4,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x0dd94b0324932858aaccaecaa2f1ebb336a2e098d535...,61384612,1725386855,{'web3_type': <class 'web3._utils.datatypes.Po...
5,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xa41c907dedcee7d8e4f59ce53c3823aa6ec4e73fc15c...,61386834,1725391821,{'web3_type': <class 'web3._utils.datatypes.Po...
6,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x2b0a866363b4aabfe76f6c5e679e74aa8b61ce247c85...,61388160,1725394753,{'web3_type': <class 'web3._utils.datatypes.Po...
7,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x917933899c6a5f8e37f31e19f92cdbff7e8ff0e2,0x197ac382d39ce862d987979bbcc9a7533e64523e5c28...,61388525,1725395575,{'web3_type': <class 'web3._utils.datatypes.Po...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x719ed60496788b0ee7726bb370ed041b72820bfb7a33d3fcdb54ece3612b72a2',
  'blk_num': 61381760,
  'timestamp': 1725380635,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0x4130ab14dF57fda204baE64251EE3BEd2C4047d5',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'CTM',
   'fee': 100}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x3f6f4f30ee4e0efc203618ec0d36d6bff0ca0d2da0301f6361dd5ae84132822a',
  'blk_num': 61383670,
  'timestamp': 1725384795,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0xc2132D05D31c914a87C6611C10748AEb04B58e8F',
   'token1': '0xd